<a href="https://colab.research.google.com/github/Rafe2001/CareerNavigator/blob/main/Study_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'student-studeis-recommendation:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4715442%2F8006400%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T061457Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D47ed98b4e050f8de648f26f301d0011fb7d11fa83183438c43ede76c8ce3f1b747a6c176b573a3490adebcff9825a9104704a0d8dcc6fe6a174158b35762198e16af031bd6447bfc4c1da18650413bb37f1c04d232c1e90cb8d3fdca8d00a249987403dc9f64c310c4215e71d5299033bfceb6911426a10cbf4d3403f94393f86fac628d65d2b81dafb3de142dee4f8b4079a4d57f68488341ea0e4ba6dad88213de1c007242beb61ff9b310b355393f5bfe280cfb01eea108c0bb30e017f80f8264a63d3cf79b5e52d74980dd3d7712212155a88ae891154eb6647bcbbd724a9e36793cd08abee36e0dabd4dd1bca0d82ecdabac88c24d529c05e7b77ea3a20'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Data Wrangling

In [ ]:
train_df = pd.read_csv("/kaggle/input/student-studeis-recommendation/student-scores.csv")
train_df.head(10)

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.duplicated().sum()

In [ ]:
cat_cols = train_df.select_dtypes(include='object').columns
cat_cols

In [ ]:
train_df.drop(columns = ['id','first_name', 'last_name', 'email'], axis = 1, inplace=True)

In [ ]:
train_df['total_score'] = train_df['math_score'] + train_df['history_score'] + train_df['physics_score'] + train_df['chemistry_score'] + train_df['biology_score'] + train_df['english_score'] + train_df['geography_score']
train_df['average_score'] = train_df['total_score'] / 7
train_df.head()

In [ ]:
train_df['career_aspiration'].value_counts()

In [ ]:
len(train_df['career_aspiration'].unique())

In [ ]:
cols = ['gender','part_time_job','extracurricular_activities', 'career_aspiration']

for col in cols:
    train_df[col] = LabelEncoder().fit_transform(train_df[col])

In [ ]:
train_df['career_aspiration'].value_counts()

In [ ]:
train_df.head(10)

In [ ]:
train_df.columns

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 42)

X = train_df.drop("career_aspiration", axis=1)
y = train_df['career_aspiration']

X_resampled ,  y_resampled = smote.fit_resample(X,y)

In [ ]:
X_resampled.shape

In [ ]:
y_resampled.shape

# Training and Evaluating the Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size= 0.2, random_state=32)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def model_train(model,X_train, X_test, y_train, y_test):
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_pred, y_test)
    return acc, model

In [ ]:
classifiers = {
    'Random Forest': RandomForestClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier(),
    'CatBoost': CatBoostClassifier(),
    'LightGBM': LGBMClassifier()
}

In [ ]:
accuracy_scores = {}
for name, classifier in classifiers.items():
    acc, _ = model_train(classifier, X_train, X_test, y_train, y_test)
    accuracy_scores[name] = acc

# Sort dictionary by values (accuracy scores) in descending order
sorted_accuracy_scores = {k: v for k, v in sorted(accuracy_scores.items(), key=lambda item: item[1], reverse=True)}

# Print sorted accuracy scores
for name, acc in sorted_accuracy_scores.items():
    print(f"{name}: {acc}")

In [ ]:
best_classifier_name, best_classifier = next(iter(sorted_accuracy_scores.items()))

In [ ]:
best_classifier_name

In [ ]:
if best_classifier_name == 'Random Forest':
    # Define hyperparameter grid for RandomizedSearchCV
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    # Initialize RandomForestClassifier
    best_classifier = RandomForestClassifier()

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(estimator=best_classifier, param_distributions=param_grid, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)

    # Get the best parameters and best score
    best_classifier = random_search.best_estimator_

In [ ]:
best_classifier

In [ ]:
best_model = CatBoostClassifier()
best_model.fit(X_train, y_train)

In [ ]:
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_pred, y_test)
print(f"{acc*100}")

In [ ]:
print(classification_report(y_pred, y_test))

In [ ]:
print("Predicted_labels:", best_model.predict(X_test.iloc[[4]]))

In [ ]:
print("Actual:", y_test.iloc[4])

# Saving model

In [ ]:
import pickle

pickle.dump(best_model, open('model.pkl','wb'))

In [ ]:
model = pickle.load(open("model.pkl", 'rb'))

# Recommendation System

In [ ]:
professions = [
    "Accountant",
    "Artist",
    "Banker",
    "Business Owner",
    "Construction Engineer",
    "Designer",
    "Doctor",
    "Game Developer",
    "Government Officer",
    "Lawyer",
    "Real Estate Developer",
    "Scientist",
    "Software Engineer",
    "Stock Investor",
    "Teacher",
    "Unknown",
    "Writer"
]

In [ ]:
def recommendation(gender, part_time_job, absence_days, extracurricular_activities,
       weekly_self_study_hour, math_score,history_score, physics_score, chemistry_score, biology_score,
       english_score, geography_score, total_score, average_score):
    gender_encoded = 1 if gender.lower() =='male' else 0
    part_time_job_encoded = 1 if part_time_job else 0
    extracurricular_activities_encoded = 1 if extracurricular_activities else 0

    feature_array = np.array([[gender_encoded, part_time_job_encoded, absence_days, extracurricular_activities_encoded,
                               weekly_self_study_hour, math_score, history_score, physics_score, chemistry_score,
                               biology_score, english_score, geography_score, total_score, average_score]])

    probabilities = model.predict_proba(feature_array)
    top_classes_idx = np.argsort(-probabilities[0])[:5]
    top_classes_names_probs = [(professions[idx], probabilities[0][idx]) for idx in top_classes_idx]
    return top_classes_names_probs

In [ ]:
gender = 'female'
part_time_job = True
absence_days = 2
extracurricular_activities = False
weekly_self_study_hour = 4
math_score = 67
history_score = 50
physics_score = 67
chemistry_score = 56
biology_score = 40
english_score = 80
geography_score = 0
total_score = 583
average_score = 83.2456

final_recommendation = recommendation(gender, part_time_job, absence_days, extracurricular_activities,
                                      weekly_self_study_hour, math_score, history_score, physics_score,
                                      chemistry_score, biology_score, english_score, geography_score,
                                      total_score, average_score)

# Print the top recommendations
print("Top Recommendations")
print("-" * 50)
for name, prob in final_recommendation:
    print(f"{name} with probability {prob*100}")